In [105]:
import pandas as pd
import numpy as np

In [106]:
### identify where to find the data 

excelFileName = 'Credit Card Comparison Data.xlsx'
# sheet names 
sheetName_Cards = 'Card Data'
sheetName_Spending = 'Personal Spending'

## column names
# card sheet
name_col = 'Card Name'
company_col = 'Credit Card Company'
owned_col = 'Owned'
category_cols = ['Dining','Travel','Flights','Groceries','Online Shopping','Entertainment','Other']
fee_col = 'Annual Fee'
bonus_col = 'Sign-Up Bonus'
benefitsFirstYear_col = 'Additional Benefits - 1st Year Only (Personal Valuation)'
benefitsEveryYear_col = 'Additional Benefits - Every Year (Personal Valuation)'
reward_col = 'Reward Type'
cpp_col = 'Cents Per Point'
rotating_col = 'Rotating'
rotatingCategories_col = 'Rotating Categories'
standardRate_col = 'Standard Rate'
spendingRequirement_col = 'Sign Up Spending Requirement'

# spending sheet
annualSpending_col = 'Annual Spending'

In [107]:
### read in data 
cardData_Raw = pd.read_excel(excelFileName, sheet_name = sheetName_Cards, index_col = name_col)
spendingData = pd.read_excel(excelFileName, sheet_name = sheetName_Spending, index_col = 0)

## get the sorted list of unique credit card companies
companies = np.sort(np.unique(cardData_Raw[company_col].values))
isCoverage = ['Coverage' in index for index in spendingData_Raw.index]
companyCoverage = spendingData_Raw.loc[isCoverage,:]
companyCoverage = companyCoverage.sort_index()
companyCoverage['Companies'] = companies

## identify cards based on reward type (cash back vs rewards)
isCashBackCard = [rewardType == 'Cash Back' for rewardType in cardData_Raw[reward_col]]
cashBackCards = cardData_Raw.index[isCashBackCard]
isRewardsCard = [rewardType == 'Rewards' for rewardType in cardData_Raw[reward_col]]
rewardsCards = cardData_Raw.index[isRewardsCard]

### convert reward points to cent valuations 
# store adjusted points in new dataframe 
cardData = cardData_Raw.copy()

centsPerPoint = pd.DataFrame()
for c in category_cols:
    centsPerPoint[c] = cardData.loc[rewardsCards, cpp_col]
cardData.loc[rewardsCards, category_cols] = cardData.loc[rewardsCards, category_cols] * centsPerPoint
cardData.loc[rewardsCards, bonus_col] = cardData.loc[rewardsCards, bonus_col] * cardData.loc[rewardsCards, cpp_col]

In [108]:
### useful variables to store
numCards = len(cardData.index)
numCategories = len(category_cols)

isOwned = [x == 'Yes' for x in cardData[owned_col]] 
ownedCards = cardData.index[isOwned] 
notOwnedCards = cardData.index[np.logical_not(isOwned)]
numNotOwnedCards = len(notOwnedCards)

In [109]:
### adjust rewards dataframe for cards that have a rotating quarter
def adjustForRotatingCards(currRewards, cards):

    # identify which cards are rotating 
    isRotating = [x != 'No' for x in cardData.loc[cards,rotating_col]]
    rotatingCards = cards[isRotating]

    # adjust rewards based on rotating type 
    for card in rotatingCards: 
        rotatingCategories = cardData.loc[card,rotatingCategories_col].split(', ')
        standardRates = cardData.loc[card,standardRate_col].split(', ')
        # reduce reward to apply for 1 quarter and adjust remaining 3 quarters to standard rate
        if cardData.loc[card,rotating_col] == 'Quarter':
            for i in range(len(rotatingCategories)):
                currRewards.loc[card, rotatingCategories[i]] = currRewards.loc[card, rotatingCategories[i]] / 4 + \
                currRewards.loc[card, rotatingCategories[i]] / 4 / cardData.loc[card, rotatingCategories[i]] * float(standardRates[i]) * 3

    return currRewards

In [110]:
### adjust rewards dataframe for cards based on company coverage (weighted by dollar spend, not simply vendor acceptance)
def adjustForCoverage(currRewards, cards):

    coverage = pd.DataFrame(np.zeros((len(cards), numCategories)))

    for i in range(len(cards)):
        currCard = cards[i]
        currCompany = cardData.loc[currCard, company_col]
        currCompanyCoverage = companyCoverage.loc[[currCompany == company for company in companyCoverage['Companies']],category_cols]
        coverage.iloc[i,:] = currCompanyCoverage.values

    coverage.index = cards
    coverage.columns = category_cols

    currRewards = currRewards * coverage
    
    return currRewards

In [111]:
### takes list of card names and returns the net value for the first year = (rewards + sign-up bonus + valuation of additional benefits - annual fee) 
### and the associated components of this calculation for every card
def getValue(cards):

    # rewards for every card in each category based on spending behavior
    categoryRewards = cardData.loc[cards,category_cols] * (spendingData.loc[annualSpending_col])

    # adjust based on rotating cards
    categoryRewards = adjustForRotatingCards(categoryRewards, cards)
    # adjust for credit card company coverage
    categoryRewards = adjustForCoverage(categoryRewards, cards)

    # reward for each card, aggregated across spending categories
    rewards = categoryRewards.sum(axis = 1)

    # additional data for value calculation
    signUpBonus = cardData.loc[cards, bonus_col]
    additionalBenefitsFirstYear = cardData.loc[cards, benefitsFirstYear_col]
    additionalBenefitsEveryYear = cardData.loc[cards, benefitsEveryYear_col]
    annualFee = cardData.loc[cards, fee_col]

    # compute value for the cards
    value = rewards + signUpBonus  + additionalBenefitsFirstYear + additionalBenefitsEveryYear - annualFee
    # store calculations in dataframe 
    output = pd.DataFrame(np.transpose([value, rewards, signUpBonus, additionalBenefitsFirstYear, additionalBenefitsEveryYear, annualFee]), \
                          index = cards, columns = ['Value','Rewards', bonus_col, benefitsFirstYear_col, benefitsEveryYear_col, fee_col])
    
    # append category rewards 
    output[categoryRewards.columns] = categoryRewards
    # sort cards in decreasing order of value
    output = output.sort_values(by=['Value'], ascending = False)

    return output

In [112]:
getValue(ownedCards)

,Value,Rewards,Sign-Up Bonus,Additional Benefits - 1st Year Only (Personal Valuation),Additional Benefits - Every Year (Personal Valuation),Annual Fee,Dining,Travel,Flights,Groceries,Online Shopping,Entertainment,Other
Card Name,,,,,,,,,,,,,
Bank of America Cash Rewards,564.131809,364.131809,200.0,0.0,0.0,0.0,50.098199,14.818123,19.539981,33.813558,214.237726,4.98826,26.635962
Uber Visa,498.312486,373.312486,100.0,0.0,25.0,0.0,150.294597,44.454370,58.619943,16.906779,71.412575,4.98826,26.635962
Discover Student,496.949090,246.949090,0.0,250.0,0.0,0.0,85.166938,14.818123,19.539981,27.050847,71.412575,4.98826,23.972365


In [113]:
getValue(notOwnedCards)

,Value,Rewards,Sign-Up Bonus,Additional Benefits - 1st Year Only (Personal Valuation),Additional Benefits - Every Year (Personal Valuation),Annual Fee,Dining,Travel,Flights,Groceries,Online Shopping,Entertainment,Other
Card Name,,,,,,,,,,,,,
Amex Plat,2063.010064,543.010064,1500.0,0.0,570.0,550.0,90.176758,29.636246,195.399810,27.050847,142.825151,9.976521,47.944731
Amex Gold,1791.532954,816.532954,1000.0,0.0,225.0,250.0,360.707033,29.636246,117.239886,108.203387,142.825151,9.976521,47.944731
Amex Green,1504.476149,704.476149,900.0,0.0,50.0,150.0,270.530275,88.908739,117.239886,27.050847,142.825151,9.976521,47.944731
Chase Reserve,1463.296851,653.296851,875.0,97.5,387.5,550.0,263.015545,77.795147,102.584900,29.586864,124.972007,8.729455,46.612933
Chase Pref,1263.748600,421.248600,875.0,37.5,25.0,95.0,146.119747,43.219526,56.991611,24.655720,104.143339,7.274546,38.844111
Capital One Savor,686.566037,386.566037,300.0,95.0,0.0,95.0,200.392796,14.818123,19.539981,33.813558,71.412575,19.953041,26.635962
Chase Freedom Unlimited,544.099819,306.599819,200.0,37.5,0.0,0.0,75.147299,22.227185,29.309972,25.360169,107.118863,7.482390,39.953942
Amex Blue Cash Preferred,534.688433,329.688433,300.0,0.0,0.0,95.0,45.088379,44.454370,58.619943,81.152540,71.412575,4.988260,23.972365
Amex Cash Magnet,490.017662,290.017662,200.0,0.0,0.0,0.0,67.632569,22.227185,29.309972,20.288135,107.118863,7.482390,35.958548


In [114]:
### for the list of cards provided, computes the reward for the card(s) that have the best reward in each category
def compareCards(cards):

    # remove rotating cards
    isRotating = [x != 'No' for x in cardData.loc[cards,rotating_col]]
    print(cards[isRotating].values + ' have been removed because their rotating status interferes with these calculations')
    cards = cards[np.logical_not(isRotating)]
    
    # best category rewards
    categoryMaxReward = cardData.loc[cards,category_cols].max()

    # identify best cards in each category
    cardsMaxReward = [''] * numCategories
    # matrix to identify best cards in each category
    isBestCard = np.zeros((len(cards),numCategories))
    for i in range(numCategories):
        for j in range(len(cards)):
            currCard = cards[j]
            currCategory = category_cols[i]
            if cardData.loc[currCard,currCategory] == categoryMaxReward[i]:
                # formatting array depending on whether a best card has been identified already
                if cardsMaxReward[i] == '':
                    cardsMaxReward[i] = currCard
                else:
                    cardsMaxReward[i] = cardsMaxReward[i] + ', ' + currCard
                isBestCard[j][i] = 1

    # compute rewards for each category based on best card in each category
    bestCards = pd.DataFrame(np.array([categoryMaxReward, cardsMaxReward]), index = ['Best Reward','Best Card'], columns = category_cols)
    bestCardsRewards = isBestCard * cardData.loc[cards,category_cols] * spendingData.loc[annualSpending_col]

    return bestCardsRewards

In [115]:
compareCards(cardData.index)

['Discover Student have been removed because their rotating status interferes with these calculations']


,Dining,Travel,Flights,Groceries,Online Shopping,Entertainment,Other
Card Name,,,,,,,
Bank of America Cash Rewards,0.000000,0.000000,0.00000,0.000000,214.237726,0.000000,0.000000
Capital One Savor One,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Capital One Savor,0.000000,0.000000,0.00000,0.000000,0.000000,19.953041,0.000000
TD Cash,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Amex Blue Cash Everyday,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Amex Blue Cash Preferred,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Amex Cash Magnet,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Chase Freedom Unlimited,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Citi Double Cash,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,53.271923


In [116]:
### determine incremental value of all unowned cards based on the cards you currently own
def getIncrementalValue():

#     reward = np.zeros((numNotOwnedCards, numCategories))
#     for i in range(numNotOwnedCards):
#         currNotOwnedCard = cardData.index[np.logical_not(isOwned)][i]
#         reward[i] = compareCards(np.append(ownedCards,currNotOwnedCard)).loc[currNotOwnedCard]

    # compute marginal reward of each card given the rewards of the cards that I currently own
#     reward = pd.DataFrame(reward, index = cardData.index[np.logical_not(isOwned)], columns = category_cols)
    reward = getValue(notOwnedCards)[category_cols]
#     marginalValue = reward - compareCards(ownedCards).max()
    marginalRewards = reward - getValue(ownedCards)[category_cols].max()
    isWorse = marginalRewards < 0
    isWorse
    marginalRewards[isWorse] = -1

    # each card's total reward across all categories
    totalReward = np.sum(marginalRewards[np.logical_not(isWorse)], axis = 1)
    
    # populate dataframe  
    marginalVal_Year1_col = 'Marginal Value - 1st Year'
    marginalVal_Year2_col = 'Marginal Value - 2nd Year'
    totalRewards_col = 'Total Rewards'
    marginalValue = pd.DataFrame(np.zeros((numNotOwnedCards,2)), index = notOwnedCards, \
                                 columns = [marginalVal_Year1_col, marginalVal_Year2_col])
    marginalValue[bonus_col] = cardData.loc[notOwnedCards, bonus_col]
    marginalValue[benefitsFirstYear_col] = cardData.loc[notOwnedCards, benefitsFirstYear_col]
    marginalValue[benefitsEveryYear_col] = cardData.loc[notOwnedCards, benefitsEveryYear_col]
    marginalValue[fee_col] = cardData.loc[notOwnedCards, fee_col]
    marginalValue[totalRewards_col] = totalReward
    marginalValue[category_cols] = marginalRewards
    marginalValue[marginalVal_Year1_col] = marginalValue[totalRewards_col] + marginalValue[bonus_col] + marginalValue[benefitsFirstYear_col] + marginalValue[benefitsEveryYear_col] - marginalValue[fee_col]
    marginalValue[marginalVal_Year2_col] = marginalValue[marginalVal_Year1_col] - (marginalValue[bonus_col] + marginalValue[benefitsFirstYear_col])
    marginalValue[spendingRequirement_col] = cardData[spendingRequirement_col]
    
    # sort values in descending marginal value for the 1st year
    marginalValue = marginalValue.sort_values(marginalVal_Year1_col, ascending = False)
    # round elements to integers
    marginalValue = marginalValue.round(0)
    
    return marginalValue

In [117]:
# # write to excel file
# outputFileName = 'Incremental Value Output.xlsx'
# getIncrementalValue().to_excel(outputFileName)

In [118]:
getIncrementalValue()

,Marginal Value - 1st Year,Marginal Value - 2nd Year,Sign-Up Bonus,Additional Benefits - 1st Year Only (Personal Valuation),Additional Benefits - Every Year (Personal Valuation),Annual Fee,Total Rewards,Dining,Travel,Flights,Groceries,Online Shopping,Entertainment,Other,Sign Up Spending Requirement
Card Name,,,,,,,,,,,,,,,
Amex Plat,1683.0,183.0,1500.0,0.0,570.0,550,163.0,-1.0,-1.0,137.0,-1.0,-1.0,5.0,21.0,5000
Amex Gold,1345.0,345.0,1000.0,0.0,225.0,250,370.0,210.0,-1.0,59.0,74.0,-1.0,5.0,21.0,4000
Amex Green,1050.0,150.0,900.0,0.0,50.0,150,250.0,120.0,44.0,59.0,-1.0,-1.0,5.0,21.0,2000
Chase Reserve,1024.0,51.0,875.0,98.0,388.0,550,214.0,113.0,33.0,44.0,-1.0,-1.0,4.0,20.0,4000
Chase Pref,857.0,-56.0,875.0,38.0,25.0,95,14.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,12.0,4000
Capital One Savor,365.0,-30.0,300.0,95.0,0.0,95,65.0,50.0,-1.0,-1.0,0.0,-1.0,15.0,0.0,3000
Chase Freedom Unlimited,253.0,16.0,200.0,38.0,0.0,0,16.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,13.0,500
Amex Blue Cash Preferred,252.0,-48.0,300.0,0.0,0.0,95,47.0,-1.0,0.0,0.0,47.0,-1.0,0.0,-1.0,1000
Amex Cash Magnet,212.0,12.0,200.0,0.0,0.0,0,12.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,9.0,1000


In [119]:
getIncrementalValue().sort_values('Marginal Value - 2nd Year', ascending = False)

,Marginal Value - 1st Year,Marginal Value - 2nd Year,Sign-Up Bonus,Additional Benefits - 1st Year Only (Personal Valuation),Additional Benefits - Every Year (Personal Valuation),Annual Fee,Total Rewards,Dining,Travel,Flights,Groceries,Online Shopping,Entertainment,Other,Sign Up Spending Requirement
Card Name,,,,,,,,,,,,,,,
Amex Gold,1345.0,345.0,1000.0,0.0,225.0,250,370.0,210.0,-1.0,59.0,74.0,-1.0,5.0,21.0,4000
Amex Plat,1683.0,183.0,1500.0,0.0,570.0,550,163.0,-1.0,-1.0,137.0,-1.0,-1.0,5.0,21.0,5000
Amex Green,1050.0,150.0,900.0,0.0,50.0,150,250.0,120.0,44.0,59.0,-1.0,-1.0,5.0,21.0,2000
Chase Reserve,1024.0,51.0,875.0,98.0,388.0,550,214.0,113.0,33.0,44.0,-1.0,-1.0,4.0,20.0,4000
Citi Double Cash,32.0,32.0,0.0,0.0,0.0,0,32.0,-1.0,-1.0,-1.0,0.0,-1.0,5.0,27.0,0
Chase Freedom Unlimited,253.0,16.0,200.0,38.0,0.0,0,16.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,13.0,500
Amex Cash Magnet,212.0,12.0,200.0,0.0,0.0,0,12.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,9.0,1000
Capital One Savor One,160.0,10.0,150.0,0.0,0.0,0,10.0,0.0,-1.0,-1.0,0.0,-1.0,10.0,0.0,500
Amex Blue Cash Everyday,207.0,7.0,200.0,0.0,0.0,0,7.0,-1.0,-1.0,-1.0,7.0,-1.0,0.0,-1.0,1000
